In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_json('./input-datasets/1633527666.ndjson',lines=True)
df2 = pd.read_json('./input-datasets/1633531868.ndjson',lines=True)
df3 = pd.read_json('./input-datasets/1633534868.ndjson',lines=True)
df4 = pd.read_json('./input-datasets/1633539068.ndjson',lines=True)
df5 = pd.read_json('./input-datasets/1633542067.ndjson',lines=True)
df6 = pd.read_json('./input-datasets/1633546872.ndjson',lines=True)

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5,df6])
df.to_csv("CSV.csv",index=False)

In [ ]:
df

In [ ]:
result = pd.read_csv("CSV.csv")
result

In [ ]:
df2 = result.drop(['coordinates', 'attribution', 
                    'location', 'mobile', 'sourceType'], axis=1)

In [ ]:
df2

In [ ]:
array = ['GB', 'FR', 'NL']
df3 = df2.loc[df2['country'].isin(array)]

In [ ]:
df = pd.read_csv('./Country.csv')
df['country'].unique()


In [ ]:
df3.to_csv('./Country.csv', index = False)

In [4]:
country = pd.read_csv('./Country.csv')

In [7]:
country

,date,parameter,value,unit,city,country,sourceName
0,"{'utc': '2021-10-06T18:00:00.000Z', 'local': '2021-10-06T19:00:00+01:00'}",pm25,1.0,µg/m³,Auchencorth,GB,DEFRA
1,"{'utc': '2021-10-06T18:00:00.000Z', 'local': '2021-10-06T19:00:00+01:00'}",pm10,2.0,µg/m³,Auchencorth,GB,DEFRA
2,"{'utc': '2021-10-06T18:00:00.000Z', 'local': '2021-10-06T19:00:00+01:00'}",o3,62.0,µg/m³,Bush Estate,GB,DEFRA
3,"{'utc': '2021-10-06T18:00:00.000Z', 'local': '2021-10-06T19:00:00+01:00'}",no2,3.0,µg/m³,Bush Estate,GB,DEFRA
4,"{'utc': '2021-10-06T18:00:00.000Z', 'local': '2021-10-06T19:00:00+01:00'}",no2,14.0,µg/m³,Dumbarton,GB,DEFRA
...,...,...,...,...,...,...,...
22954,"{'utc': '2021-10-06T08:00:00.000Z', 'local': '2021-10-06T08:00:00+00:00'}",pm25,4.0,µg/m³,Reading,GB,London Air Quality Network
22955,"{'utc': '2021-10-06T09:00:00.000Z', 'local': '2021-10-06T09:00:00+00:00'}",pm25,3.0,µg/m³,Reading,GB,London Air Quality Network
22956,"{'utc': '2021-10-06T10:00:00.000Z', 'local': '2021-10-06T10:00:00+00:00'}",pm25,5.0,µg/m³,Reading,GB,London Air Quality Network
22957,"{'utc': '2021-10-06T11:00:00.000Z', 'local': '2021-10-06T11:00:00+00:00'}",pm25,3.0,µg/m³,Reading,GB,London Air Quality Network


In [ ]:
country.drop('averagingPeriod', inplace=True, axis = 1)

In [8]:
country['date'].info

<bound method Series.info of 0        {'utc': '2021-10-06T18:00:00.000Z', 'local': '...
1        {'utc': '2021-10-06T18:00:00.000Z', 'local': '...
2        {'utc': '2021-10-06T18:00:00.000Z', 'local': '...
3        {'utc': '2021-10-06T18:00:00.000Z', 'local': '...
4        {'utc': '2021-10-06T18:00:00.000Z', 'local': '...
                               ...                        
22954    {'utc': '2021-10-06T08:00:00.000Z', 'local': '...
22955    {'utc': '2021-10-06T09:00:00.000Z', 'local': '...
22956    {'utc': '2021-10-06T10:00:00.000Z', 'local': '...
22957    {'utc': '2021-10-06T11:00:00.000Z', 'local': '...
22958    {'utc': '2021-10-06T12:00:00.000Z', 'local': '...
Name: date, Length: 22959, dtype: object>

In [ ]:
import json
def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("'", '"')
    return json.loads(dict_string)

In [ ]:
country.date = country.date.apply(string_to_dict)

In [ ]:
full_date = [obj['utc'] for obj in country['date']]
full_date[0]

In [ ]:
from datetime import *
date = []
hour = []
for i in range(len(country['date'])):
    dt_time = datetime.strptime(full_date[i], "%Y-%m-%dT%H:%M:%S.%f%z")
    date.append(str(dt_time.date()))
    hour.append(str(dt_time.time()))

In [ ]:
len(country['date']) == len(hour)

In [ ]:
country = country.assign(hours=hour)
country = country.assign(dates = date)

In [ ]:
country.drop('date', inplace=True, axis = 1)

In [16]:
ct = pd.read_csv('./final_csv.csv')

In [27]:
ct['parameter'].value_counts()

no2     9303
pm10    6180
o3      3541
pm25    2895
so2      863
co       177
Name: parameter, dtype: int64

In [28]:
dff = ct.groupby(['city','parameter', 'country', 'unit','dates'])['value'].mean().reset_index()
dff['parameter'].value_counts()

no2     252
pm10    218
o3      179
pm25    171
so2      63
co       13
Name: parameter, dtype: int64

Transform

In [4]:
dhour = pd.read_csv('../Utils/recources/hourly_value_metrics.csv')

In [5]:
dhour['parameter'].value_counts()

no2     254
pm10    220
o3      180
pm25    173
so2      63
co       13
Name: parameter, dtype: int64

In [6]:
def get_aqi_category(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Moderate"
    elif x <= 150:
        return "Unhealthy for Sensitive Groups"
    elif x <= 200:
        return "Unhealthy"
    elif x <= 300:
        return "Very Unhealthy"
    elif x > 300:
        return "Hazardous"
    else:
        return None

In [7]:
import aqi
pm25_value = dhour.query('parameter == "pm25" & value > 0')
pm25 = list()
for x in pm25_value['value']:
    value = aqi.to_iaqi(aqi.POLLUTANT_PM25, x, algo=aqi.ALGO_EPA)
    pm25.append(get_aqi_category(value))
aqi_pm25 = pm25_value.assign(pm25_aqi = pm25)
aqi_pm25

,city,country,parameter,sourceName,dates,unit,value,pm25_aqi
5,Ain,FR,pm25,EEA France,2021-10-06,µg/m³,3.100000,Good
9,Aisne,FR,pm25,EEA France,2021-10-06,µg/m³,5.191667,Good
12,Alpes-Maritimes,FR,pm25,EEA France,2021-10-06,µg/m³,5.000000,Good
16,Alpes-de-Haute-Provence,FR,pm25,EEA France,2021-10-06,µg/m³,1.581250,Good
21,Amsterdam,NL,pm25,Netherlands,2021-10-06,µg/m³,4.478571,Good
...,...,...,...,...,...,...,...,...
876,Wrexham,GB,pm25,DEFRA,2021-10-06,µg/m³,3.400000,Good
883,Yonne,FR,pm25,EEA France,2021-10-06,µg/m³,4.716667,Good
886,York,GB,pm25,DEFRA,2021-10-06,µg/m³,2.000000,Good
890,Yorkshire & Humberside,GB,pm25,DEFRA,2021-10-06,µg/m³,5.250000,Good


In [8]:
pm10_value = dhour.query('parameter == "pm10" & value > 0')
pm10 = list()

for x in pm10_value['value']:
    value = aqi.to_iaqi(aqi.POLLUTANT_PM10, x, algo=aqi.ALGO_EPA)
    pm10.append(get_aqi_category(value))
aqi_pm10 = pm10_value.assign(pm10_aqi = pm10)
aqi_pm10

,city,country,parameter,sourceName,dates,unit,value,pm10_aqi
4,Ain,FR,pm10,EEA France,2021-10-06,µg/m³,7.041176,Good
8,Aisne,FR,pm10,EEA France,2021-10-06,µg/m³,6.900000,Good
11,Alpes-Maritimes,FR,pm10,EEA France,2021-10-06,µg/m³,23.356250,Good
15,Alpes-de-Haute-Provence,FR,pm10,EEA France,2021-10-06,µg/m³,4.950000,Good
20,Amsterdam,NL,pm10,Netherlands,2021-10-06,µg/m³,13.256250,Good
...,...,...,...,...,...,...,...,...
885,York,GB,pm10,DEFRA,2021-10-06,µg/m³,5.300000,Good
888,Yorkshire,GB,pm10,DEFRA,2021-10-06,µg/m³,9.600000,Good
894,Zaandam,NL,pm10,Netherlands,2021-10-06,µg/m³,10.750000,Good
897,Zaanstad,NL,pm10,Netherlands,2021-10-06,µg/m³,13.350000,Good


In [9]:
#aqi_pm10.merge(aqi_pm25, how='left', on='city')
aqi_category = pd.concat([aqi_pm25,aqi_pm10]).fillna('No')

In [18]:
g = pd.concat([aqi_category,dhour], ignore_index=True).drop_duplicates(subset='value').fillna('No')
g.query('city == "Amsterdam"')

,city,country,parameter,sourceName,dates,unit,value,pm25_aqi,pm10_aqi
4,Amsterdam,NL,pm25,Netherlands,2021-10-06,µg/m³,4.478571,Good,No
165,Amsterdam,NL,pm10,Netherlands,2021-10-06,µg/m³,13.256250,No,Good
388,Amsterdam,NL,co,Netherlands,2021-10-06,µg/m³,0.335200,No,No
389,Amsterdam,NL,no2,Netherlands,2021-10-06,µg/m³,16.709091,No,No
390,Amsterdam,NL,o3,Netherlands,2021-10-06,µg/m³,57.400000,No,No
393,Amsterdam,NL,so2,Netherlands,2021-10-06,µg/m³,0.300000,No,No


In [15]:
g

,city,country,parameter,sourceName,dates,unit,value,pm25_aqi,pm10_aqi
0,Ain,FR,pm25,EEA France,2021-10-06,µg/m³,3.100000,Good,No
1,Aisne,FR,pm25,EEA France,2021-10-06,µg/m³,5.191667,Good,No
2,Alpes-Maritimes,FR,pm25,EEA France,2021-10-06,µg/m³,5.000000,Good,No
3,Alpes-de-Haute-Provence,FR,pm25,EEA France,2021-10-06,µg/m³,1.581250,Good,No
4,Amsterdam,NL,pm25,Netherlands,2021-10-06,µg/m³,4.478571,Good,No
...,...,...,...,...,...,...,...,...,...
1263,Yvelines,FR,o3,EEA France,2021-10-06,µg/m³,66.446667,No,No
1264,Zaandam,NL,no2,Netherlands,2021-10-06,µg/m³,5.700000,No,No
1269,Zegveld,NL,no2,Netherlands,2021-10-06,µg/m³,3.605000,No,No
1270,Zegveld,NL,o3,Netherlands,2021-10-06,µg/m³,72.770000,No,No


In [ ]:
country.assign(hours=hour)

In [ ]:
df = country.pop('hours') # remove column b and store it in df1
df2 = country.pop('dates') # remove column x and store it in df2
df['b']=df1 # add b series as a 'new' column.
df['x']=df2 # add b series as a 'new' column.

In [19]:
import pandas as pd
import sqlite3 as db

In [20]:
conn = db.connect('sqlite.db')

In [22]:
g.to_sql('Air_quality', conn, if_exists='replace')

731

In [23]:
pd.read_sql_query('select * from Air_quality', conn)

,index,city,country,parameter,sourceName,dates,unit,value,pm25_aqi,pm10_aqi
0,0,Ain,FR,pm25,EEA France,2021-10-06,µg/m³,3.100000,Good,No
1,1,Aisne,FR,pm25,EEA France,2021-10-06,µg/m³,5.191667,Good,No
2,2,Alpes-Maritimes,FR,pm25,EEA France,2021-10-06,µg/m³,5.000000,Good,No
3,3,Alpes-de-Haute-Provence,FR,pm25,EEA France,2021-10-06,µg/m³,1.581250,Good,No
4,4,Amsterdam,NL,pm25,Netherlands,2021-10-06,µg/m³,4.478571,Good,No
...,...,...,...,...,...,...,...,...,...,...
726,1263,Yvelines,FR,o3,EEA France,2021-10-06,µg/m³,66.446667,No,No
727,1264,Zaandam,NL,no2,Netherlands,2021-10-06,µg/m³,5.700000,No,No
728,1269,Zegveld,NL,no2,Netherlands,2021-10-06,µg/m³,3.605000,No,No
729,1270,Zegveld,NL,o3,Netherlands,2021-10-06,µg/m³,72.770000,No,No


In [44]:
import os
import json
from datetime import *
def json_to_csv(path):
    """
    It takes a path to a directory of JSON files, reads them in, and writes them out to a single CSV
    file
    
    :param path: The path to the JSON files
    """
    json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.ndjson')]
    df = pd.DataFrame()

    for index, js in enumerate(json_files):
        with open(os.path.join(path, js)) as json_file:
            df = df.append([pd.read_json(json_file,lines=True)])
    
    #df.to_csv('data.csv', index=False, encoding='utf-8')
    return df

def extract_country(csv_path):

    result = csv_path.drop(['coordinates', 'attribution',
                            'location', 'mobile', 'sourceType', 'averagingPeriod'], axis = 1)

    array = ['GB', 'FR', 'NL']
    selected_country = result.loc[result['country'].isin(array)]
    return selected_country

def extract_date(country):

    #country.date = country.date.apply(string_to_dict)
    full_date = [obj['utc'] for obj in country['date']]

    date = []
    hour = []
    for i in range(len(country['date'])):
        dt_time = datetime.strptime(full_date[i], "%Y-%m-%dT%H:%M:%S.%f%z")
        date.append(str(dt_time.date()))
        hour.append(str(dt_time.time()))
    
    country = country.assign(hours=hour)
    country = country.assign(dates = date)
    country.drop('date', inplace=True, axis = 1)
    extracted_data = country.sort_values(by = 'country')
    print(extracted_data)

def string_to_dict(dict_string):
    # Convert to proper json format
    #dict_string = dict_string.replace("'", '"')
    return json.loads(dict_string)

path = '/home/biniam/Desktop/ETL_Pipline/Utils/resources/input-datasets'
csv_path = json_to_csv(path)
country = extract_country(csv_path)

/tmp/ipykernel_182083/4027287937.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([pd.read_json(json_file,lines=True)])
/tmp/ipykernel_182083/4027287937.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([pd.read_json(json_file,lines=True)])
/tmp/ipykernel_182083/4027287937.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([pd.read_json(json_file,lines=True)])
/tmp/ipykernel_182083/4027287937.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([pd.read_json(json_file,lines=True)])
/tmp/ipykernel_182083/4027287937.py:16: FutureWarning: The frame.append method is deprec

In [45]:
date_csv = extract_date(country)

      parameter   value   unit             city country   sourceName  \
12609      pm10   20.50  µg/m³          Gironde      FR   EEA France   
9854        no2    3.20  µg/m³           Landes      FR   EEA France   
9855        no2    4.00  µg/m³           Landes      FR   EEA France   
9856        no2    2.80  µg/m³          Reunion      FR   EEA France   
9857        no2    3.80  µg/m³          Reunion      FR   EEA France   
...         ...     ...    ...              ...     ...          ...   
13333        o3 -999.00  µg/m³         Schiedam      NL  Netherlands   
13334        o3 -999.00  µg/m³        Maassluis      NL  Netherlands   
13335        o3 -999.00  µg/m³  Hoek v. Holland      NL  Netherlands   
13337        o3   46.60  µg/m³        Amsterdam      NL  Netherlands   
13353        o3   26.34  µg/m³        Dordrecht      NL  Netherlands   

          hours       dates  
12609  11:00:00  2021-10-06  
9854   13:00:00  2021-10-06  
9855   14:00:00  2021-10-06  
9856   11:00:00